In [119]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/random-video/ms_rachel.mov
/kaggle/input/random-video/bluey.mov
/kaggle/input/random-video/Diana_and_roma.mov
/kaggle/input/random-video/lalafun.mov
/kaggle/input/random-video/cocomelon.mov
/kaggle/input/random-video/Calm music.mov
/kaggle/input/random-video/calm_low_sound.mov


In [120]:
import os

# Main folders
os.makedirs("src/tools", exist_ok=True)
os.makedirs("demo", exist_ok=True)
os.makedirs("data", exist_ok=True)
os.makedirs("results", exist_ok=True)

# Make Python packages
open("src/__init__.py", "w").close()
open("src/tools/__init__.py", "w").close()

In [121]:
# Add /kaggle/working to Python path
sys.path.append("/kaggle/working")

In [122]:
import os
import cv2
import numpy as np
import librosa
import subprocess
import threading
import json

# ----------------------------
# Audio/Video Feature Extraction
# ----------------------------
def convert_to_wav(video_path, wav_path="temp_audio.wav"):
    command = [
        "ffmpeg",
        "-y",
        "-i", video_path,
        "-ar", "16000",
        "-ac", "1",
        wav_path
    ]
    subprocess.run(command, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    return wav_path

def extract_visual_features(video_path, max_frames=150):
    cap = cv2.VideoCapture(video_path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    avg_brightness = 0
    motion_sum = 0
    prev_gray = None

    for _ in range(min(frame_count, max_frames)):
        ret, frame = cap.read()
        if not ret:
            break
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        avg_brightness += np.mean(gray)
        if prev_gray is not None:
            motion_sum += np.mean(np.abs(gray - prev_gray))
        prev_gray = gray

    cap.release()
    brightness = avg_brightness / max(1, min(frame_count, max_frames))
    motion = motion_sum / max(1, min(frame_count, max_frames))
    return {"motion": float(motion), "brightness": float(brightness)}

def extract_audio_features(video_path):
    try:
        wav_path = convert_to_wav(video_path)
        y, sr = librosa.load(wav_path, sr=16000)
        tempo, _ = librosa.beat.beat_track(y=y, sr=sr)
        rms = np.mean(librosa.feature.rms(y=y))
        os.remove(wav_path)
        return {
            "bpm": float(np.squeeze(tempo)),
            "volume": float(np.squeeze(rms))
        }
    except Exception as e:
        print(f"Audio extraction failed: {e}")
        return {"bpm": 0.0, "volume": 0.0}

# ----------------------------
# Coordinator Agent
# ----------------------------
def safe_float(val):
    try:
        return float(val)
    except:
        return 0.0

def safe_bool(val):
    try:
        return bool(val)
    except:
        return False

class CoordinatorAgent:
    def __init__(self):
        pass

    def summarize_for_parent(self, features):
        summary = []
        if features.get("motion", 0.0) > 18:
            summary.append("High visual motion")
        if features.get("brightness", 0.0) > 200:
            summary.append("Bright scenes")
        if features.get("bpm", 0.0) > 140:
            summary.append("Fast-paced audio")
        if features.get("volume", 0.0) > 0.12:
            summary.append("Loud volume")
        return ", ".join(summary) if summary else "Normal content"

    def analyze_video(self, video_path):
        visual_result, audio_result = {}, {}

        def run_visual():
            nonlocal visual_result
            visual_result = extract_visual_features(video_path)
            visual_result = {k: safe_float(v) for k, v in visual_result.items()}

        def run_audio():
            nonlocal audio_result
            audio_result = extract_audio_features(video_path)
            audio_result = {k: safe_float(v) for k, v in audio_result.items()}

        # Run in parallel
        t1 = threading.Thread(target=run_visual)
        t2 = threading.Thread(target=run_audio)
        t1.start()
        t2.start()
        t1.join()
        t2.join()

        features = {**visual_result, **audio_result}

        flagged = (
            features.get("motion", 0.0) > 18 or
            features.get("brightness", 0.0) > 200 or
            features.get("bpm", 0.0) > 140 or
            features.get("volume", 0.0) > 0.12
        )

        parent_summary = self.summarize_for_parent(features)
        alternatives = suggest_alternatives(features)

        return {
            "flagged": safe_bool(flagged),
            "features": {k: safe_float(v) for k, v in features.items()},
            "summary": parent_summary,
            "alternatives": alternatives
        }

    def analyze_videos_in_folder(self, folder_path):
        results = {}
        for file in os.listdir(folder_path):
            if file.endswith((".mp4", ".mov")):
                path = os.path.join(folder_path, file)
                results[file] = self.analyze_video(path)
        return results

# ----------------------------
# USAGE EXAMPLE
# ----------------------------
if __name__ == "__main__":
    agent = CoordinatorAgent()
    video_folder = "/kaggle/input/random-video/"
    results = agent.analyze_videos_in_folder(video_folder)
    print(json.dumps(results, indent=2))

{
  "ms_rachel.mov": {
    "flagged": true,
    "features": {
      "motion": 23.74744721908703,
      "brightness": 129.5730644320605,
      "bpm": 170.45454545454547,
      "volume": 0.05656275898218155
    },
    "summary": "High visual motion, Fast-paced audio",
    "alternatives": [
      "Slower-paced video or storytime activity",
      "Soft music or calm narration"
    ]
  },
  "bluey.mov": {
    "flagged": true,
    "features": {
      "motion": 13.60134041179478,
      "brightness": 160.62838530185235,
      "bpm": 170.45454545454547,
      "volume": 0.11298386007547379
    },
    "summary": "Fast-paced audio",
    "alternatives": [
      "Soft music or calm narration"
    ]
  },
  "Diana_and_roma.mov": {
    "flagged": true,
    "features": {
      "motion": 43.49596967488086,
      "brightness": 166.8306471540318,
      "bpm": 133.92857142857142,
      "volume": 0.11145633459091187
    },
    "summary": "High visual motion",
    "alternatives": [
      "Slower-paced video o

In [123]:
import os
import json

# Folder containing your video files
video_folder = "/kaggle/input/random-video"

# Create results folder if it doesn't exist
os.makedirs("results", exist_ok=True)

# Initialize the agent
agent = CoordinatorAgent()

# Dictionary to store all results
all_results = {}

# Loop through all .mp4 or .mov files
for filename in os.listdir(video_folder):
    if filename.endswith((".mp4", ".mov")):
        video_path = os.path.join(video_folder, filename)
        result = agent.analyze_video(video_path)
        # Add a simple parent-friendly summary
        summary_parts = []
        if result["features"]["motion"] > 18:
            summary_parts.append("High visual motion")
        if result["features"]["bpm"] > 140:
            summary_parts.append("Fast-paced audio")
        if result["features"]["volume"] > 0.12:
            summary_parts.append("Loud volume")
        result["summary"] = ", ".join(summary_parts) if summary_parts else "Calm content"
        
        all_results[filename] = result

# Save all results automatically
with open("results/analysis.json", "w") as f:
    json.dump(all_results, f, indent=2)

print("All video results saved to results/analysis.json")

All video results saved to results/analysis.json
